## 数据预处理

## 1. 获取沪深300股票代码

In [1]:
import requests

#爬取沪深300股票代码
def spider_for_hs300():
    page=1
    hs300=[]
    while(page<=6):
        url = 'https://datainterface.eastmoney.com/EM_DataCenter/js.aspx?'
        key_dict = {
            'cb': 'jQuery112305522721612895851_1626420027301',
            'st': '1',
            'sr': '-1',
            'ps': '50',
            'p': page,
            'type': 'SHSZZS',
            'sty': 'SHSZZS',
            'code': '000300'
        }
        data = requests.get(url, params=key_dict)
        res=data.text
        for i in range(len(res)):
            if((i>0)&(res[i-1]=='"')&(res[i]>='0')&(res[i]<='9')):
                cnt=0
                ans=""
                while(cnt<6):
                    ans=ans+res[i]
                    cnt+=1
                    i+=1
                hs300.append(ans)
        page+=1
    return hs300

res = spider_for_hs300()
print(res)
print(len(set(res)))

['688396', '688363', '688169', '688126', '688111', '688036', '688012', '688009', '688008', '603993', '603986', '603939', '603899', '603882', '603833', '603806', '603799', '603659', '603658', '603517', '603501', '603392', '603369', '603338', '603288', '603259', '603233', '603195', '603160', '603087', '603019', '601998', '601995', '601990', '601989', '601988', '601985', '601939', '601933', '601919', '601916', '601901', '601899', '601888', '601881', '601878', '601877', '601872', '601857', '601838', '601818', '601816', '601808', '601800', '601799', '601788', '601766', '601727', '601698', '601696', '601688', '601669', '601668', '601658', '601633', '601628', '601618', '601607', '601601', '601600', '601555', '601398', '601390', '601377', '601360', '601336', '601328', '601319', '601318', '601288', '601238', '601236', '601231', '601229', '601225', '601216', '601211', '601186', '601169', '601166', '601162', '601155', '601138', '601111', '601108', '601100', '601088', '601077', '601066', '601021',

## 2. 筛出沪深300成分的股票数据

In [2]:
import pandas as pd

#从所有数据中筛选沪深300股票数据
data = pd.read_csv("./data/data.csv")
ans = pd.DataFrame(columns=('kdcode','dt','volume','turnover','adj_open','adj_close','adj_high','adj_low','adj_downprice','adj_upprice','pct_chg'))

for i in range(300):
    da = data.loc[data['kdcode']==res[i]+".SZ"]
    if da.empty:
        da = data.loc[data['kdcode']==res[i]+".SH"]
    ans = pd.concat([da, ans])

* 没有重复数据 无需去重

In [3]:
# 去重
print(len(ans))
ans.drop_duplicates(subset=['kdcode', 'dt'],keep='first',inplace=True)
print(len(ans))

# ans.to_csv("./data/hs300.csv", index=False)

94466
94466


## 3. 分割训练集、测试集

In [4]:
import re

datalist = ans.values.tolist()
trainSet = []
testSet = []

for data_i in datalist:
    if re.match('2020', data_i[1]) == None:
        trainSet.append(data_i)
    else:
        testSet.append(data_i)
        
print("训练集共 " + str(len(trainSet)) + " 条")
print("测试集共 " + str(len(testSet)) + " 条")

# trainSet = pd.DataFrame(trainSet, columns=('kdcode','dt','volume','turnover','adj_open','adj_close','adj_high','adj_low','adj_downprice','adj_upprice','pct_chg'))
# testSet = pd.DataFrame(testSet, columns=('kdcode','dt','volume','turnover','adj_open','adj_close','adj_high','adj_low','adj_downprice','adj_upprice','pct_chg'))
# trainSet.to_csv("./data/train.csv", index=False)
# testSet.to_csv("./data/test.csv", index=False)

训练集共 74561 条
测试集共 19905 条


## 4. 填补训练集停牌数据

In [10]:
import pandas as pd
from datetime import datetime
from datetime import timedelta


# 判断闰年
def isRunYear(year):
    if year % 100 == 0:
        if year % 400 == 0:
            return 1
    else:
        if year % 4 == 0:
            return 1
    return 0


# 判断是一年中的第几天
def dayNum(year, month, day):
    ret = 0
    ping = [31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]
    run = [31, 29, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]
    if isRunYear(year):
        for i in range(month - 1):
            ret = ret + run[i]
    else:
        for i in range(month - 1):
            ret = ret + ping[i]
    return ret + day


# 判断是否是周末
def isSunOrSat(year, month, day):
    S = (year + (year - 1) // 4 - (year - 1) // 100 + (year - 1) // 400) % 7  # 第year年第一天是周S
    weekday = (dayNum(year, month, day) + S - 1) % 7
    if weekday == 0 or weekday == 6:
        return 1
    else:
        return 0


# trainSet = pd.read_csv("train.csv").values.tolist()
datetime_tmp = pd.to_datetime('2022-01-01')
date_tmp = datetime.date(datetime_tmp)
data_tmp = []
trainData = []

for data_i in trainSet:
    now_date = datetime.date(pd.to_datetime(data_i[1]))
    if now_date < date_tmp:
        trainData.append(data_i)
        date_tmp = now_date
        data_tmp = data_i.copy()
    else:
        daydelta = now_date - date_tmp
        if daydelta.days != 1:
            for i in range(1, daydelta.days):
                add_day = date_tmp + timedelta(days=i)
                if isSunOrSat(add_day.year, add_day.month, add_day.day) != 1:
                    add_data = data_tmp.copy()
                    add_data[1] = str(add_day)
                    trainData.append(add_data)
        date_tmp = now_date
        data_tmp = data_i.copy()
        trainData.append(data_i)

print("填充后训练集数据共 " + str(len(trainData)) + " 条")
trainData = pd.DataFrame(trainData, columns=('kdcode', 'dt', 'volume', 'turnover', 'adj_open', 'adj_close', 'adj_high', 'adj_low', 'adj_downprice', 'adj_upprice','pct_chg'))
trainData.to_csv("./data/trainData.csv", index=False)

填充后训练集数据共 81574 条
